In [1]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=60, max_dist=9.652110123602292e-05, ..., log_cap=-0.5619000922649475)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[1.86401077+0.j         0.        +0.j         0.        +0.j        ]
 [0.40568423-0.65191208j 0.78984324+0.j         0.        +0.j        ]
 [0.02417466-0.05871347j 0.08408474+0.21475773j 0.64228612+0.j        ]] 

[[ 2.30299211+0.j          0.        +0.j          0.        +0.j        ]
 [ 1.93406064+0.37703279j  0.760699  +0.j          0.        +0.j        ]
 [-0.44726003+2.09760999j  0.37884666+0.67761631j  0.73148825+0.j        ]] 

[[ 2.11570817+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.34303547+0.77486223j  0.48111456+0.j          0.        +0.j        ]
 [-0.18472265-0.08619213j -0.40748028+0.08285216j  0.3138829 +0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.31101482271171077, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 3
#001: max_dist = 0.47140452 @ sys = 0
#002: max_dist = 0.56568542 @ sys = 1
#003: max_dist = 0.62853936 @ sys = 2
#004: max_dist = 0.58232323 @ sys = 3
#005: max_dist = 0.59305730 @ sys = 0
#006: max_dist = 0.59545834 @ sys = 1
#007: max_dist = 0.59262283 @ sys = 2
#008: max_dist = 0.59353004 @ sys = 3
#009: max_dist = 0.59357133 @ sys = 0
#010: max_dist = 0.59340661 @ sys = 1
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=25, max_dist=4.925768997189056e-07, ..., log_cap=-0.15872212095424812)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[ 0.32067+0.j      -0.0198 +0.0074j   0.00931-0.00041j]
 [-0.0198 -0.0074j   0.36438+0.j      -0.0121 -0.01212j]
 [ 0.00931+0.00041j -0.0121 +0.01212j  0.31494+0.j     ]] 

[[ 0.4+0.j  0. +0.j  0. -0.j]
 [ 0. -0.j  0.3+0.j -0. +0.j]
 [ 0. +0.j -0. -0.j  0.3+0.j]] 

[[ 0.7+0.j -0. +0.j -0. +0.j]
 [-0. -0.j  0.2+0.j -0. +0.j]
 [-0. +0.j -0. -0.j  0.1+0.j]]
